# **مجموعه داده:**

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px">

*بخش اول:*
---
مشکلات کلی که مشاهده می‌شود:

1. تفاوت در ساختار و ویژگی‌ها:
    * دیتاست فارسی شامل ستون‌های tokens، ner_tags، و ner_tags_names است، در حالی که دیتاست انگلیسی شامل ستون‌های id، tokens، pos_tags، chunk_tags و ner_tags است.
    * این تفاوت باعث می‌شود که در هنگام ادغام، نیاز به تنظیم ساختار داده‌ها باشد.

2. تفاوت در برچسب‌های ner_tags:
    * برچسب‌های ner_tags در دیتاست فارسی شامل انواعی مانند B_LOC، B_PRO و غیره است.
    * برچسب‌های دیتاست انگلیسی با نام‌های مختلف مانند B-PER، B-LOC و غیره تعریف شده‌اند. این تفاوت در نام‌گذاری می‌تواند مشکلاتی در ادغام داده‌ها ایجاد کند.

3. وجود برچسب‌های اضافی در دیتاست فارسی:
    * دیتاست فارسی برچسب‌هایی مانند B_EVE، I_EVE، B_PRO، I_PRO و غیره دارد که در دیتاست انگلیسی وجود ندارند.
    * این برچسب‌ها باید به یک مجموعه عمومی از برچسب‌ها نگاشت شوند تا مدل بتواند از آن‌ها استفاده کند.

4. عدم هم‌ترازی داده‌ها:
    * در دیتاست فارسی، ستون ner_tags_names وجود دارد، اما در دیتاست انگلیسی فقط ner_tags و کلاس‌های آن وجود دارند.

---

راه‌حل‌های پیشنهادی:

1. **استانداردسازی برچسب‌ها**: استفاده از یک نقشه کلی (general_mapping) که تمامی برچسب‌های هر دو دیتاست را به یک فضای مشترک نگاشت کند. این نقشه باید شامل تمامی برچسب‌های ممکن باشد.


2. **هماهنگ‌سازی ساختار داده‌ها**: ایجاد ستون‌های مورد نیاز (مانند ner_tags_names در دیتاست انگلیسی) یا حذف ستون‌های اضافی برای همگام‌سازی ساختار داده‌ها.

3. **حذف یا نگاشت برچسب‌های غیرمشترک**:  برچسب‌هایی که فقط در دیتاست فارسی وجود دارند، مانند B_EVE و I_PRO، باید به یک برچسب عمومی (مانند O) نگاشت شوند یا حذف شوند.

4. **تبدیل فرمت**: اگر تفاوتی در نام‌گذاری برچسب‌ها وجود دارد (مانند B-PER در دیتاست انگلیسی و B_PER در دیتاست فارسی)، نام‌ها باید استاندارد شوند.

5. **ادغام ستون‌ها**: ترکیب ستون‌های tokens و ner_tags از هر دو دیتاست در قالبی واحد که مدل بتواند به راحتی آن را پردازش کند.

6. **اعتبارسنجی داده‌ها**: حذف داده‌های ناقص یا نامعتبر پیش از ادغام نهایی.



In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset, DatasetDict, Dataset
import torch
import pandas as pd

device = 'cuda' if torch.cuda.is_available() else 'cpu'

# Load datasets
persian_dataset = load_dataset("AliFartout/PEYMA-ARMAN-Mixed")
english_dataset = load_dataset("conll2003", trust_remote_code=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(persian_dataset['train'].features)
# persian_dataset['train']['ner_tags']
print(persian_dataset['train']['ner_tags'][0])
print(persian_dataset['train']['ner_tags_names'][0])

print(english_dataset['train'].features)
# persian_dataset['train']['ner_tags']
print(english_dataset['train']['ner_tags'][2])
print(english_dataset['train']['tokens'][2])


{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'ner_tags': Sequence(feature=ClassLabel(names=['B_LOC', 'I_DAT', 'B_PCT', 'I_LOC', 'I_PER', 'I_MON', 'B_ORG', 'B_PRO', 'B_PER', 'O', 'I_PCT', 'I_ORG', 'B_FAC', 'B_DAT', 'B_TIM', 'I_TIM', 'I_EVE', 'B_MON', 'I_PRO', 'B_EVE', 'I_FAC'], id=None), length=-1, id=None), 'ner_tags_names': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}
[5, 20, 1, 20, 20, 20, 20, 1, 20, 20]
['B_PER', 'O', 'B_LOC', 'O', 'O', 'O', 'O', 'B_LOC', 'O', 'O']
{'id': Value(dtype='string', id=None), 'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None), 'pos_tags': Sequence(feature=ClassLabel(names=['"', "''", '#', '$', '(', ')', ',', '.', ':', '``', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'NN|SYM', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB'],

# **سوال اول:**

*بخش اول:*
---
# پاسخ خود را اینجا بنویسید:

In [ ]:
id_to_label_mapping_persian = {}
for tagsnames, tagslabels in zip(persian_dataset['train']['ner_tags_names'], persian_dataset['train']['ner_tags']):
  if tagsnames != None:
    for lebel, lebelid in zip(tagsnames, tagslabels):
      id_to_label_mapping_persian[lebelid] = lebel


id_to_label_mapping_english = {
    0: 'O',
    1: 'B_PER',
    2: 'I_PER',
    3: 'B_ORG',
    4: 'I_ORG',
    5: 'B_LOC',
    6: 'I_LOC',
    7: 'B_MISC',
    8: 'I_MISC'
}

print(id_to_label_mapping_persian)
print(id_to_label_mapping_english)


{5: 'B_PER', 20: 'O', 1: 'B_LOC', 12: 'I_PER', 3: 'B_ORG', 10: 'I_ORG', 16: 'B_EVE', 17: 'I_EVE', 8: 'I_LOC', 18: 'B_PRO', 14: 'B_FAC', 15: 'I_FAC', 2: 'B_MON', 9: 'I_MON', 0: 'B_DAT', 7: 'I_DAT', 19: 'I_PRO', 6: 'B_TIM', 13: 'I_TIM', 4: 'B_PCT', 11: 'I_PCT'}
{0: 'O', 1: 'B_PER', 2: 'I_PER', 3: 'B_ORG', 4: 'I_ORG', 5: 'B_LOC', 6: 'I_LOC', 7: 'B_MISC', 8: 'I_MISC'}


In [ ]:
# General mapping for NER tags
general_mapping = {
    'O': 0,
    'B_PER': 1,
    'I_PER': 2,
    'B_LOC': 3,
    'I_LOC': 4,
    'B_ORG': 5,
    'I_ORG': 6,
    'B_MISC': 7,
    'I_MISC': 8,
    'B-PER': 1,
    'I-PER': 2,
    'B-LOC': 3,
    'I-LOC': 4,
    'B-ORG': 5,
    'I-ORG': 6,
    'B-MISC': 7,
    'I-MISC': 8,
    'B_EVE': 9,
    'I_EVE': 10,
    'B_PRO': 11,
    'I_PRO': 12,
    'B_FAC': 13,
    'I_FAC': 14,
    'B_MON': 15,
    'I_MON': 16,
    'B_DAT': 17,
    'I_DAT': 18,
    'B_TIM': 19,
    'I_TIM': 20,
    'B_PCT': 21,
    'I_PCT': 22
}


# Preprocessing function
def preprocess_dataset(dataset, language, split, id_to_label_mapping, general_mapping):
    df = pd.DataFrame(dataset[split])

    df['language'] = language

    # Map `ner_tags` to generalized labels using `id_to_label_mapping` and `general_mapping`
    def map_labels(example):
        # Convert tag IDs to tag names
        tag_names = [id_to_label_mapping[tag] for tag in example]
        # Map tag names to the generalized tag space
        return [general_mapping[tag] for tag in tag_names if tag in general_mapping]

    # Apply mapping to `ner_tags`
    df['ner_tags'] = df['ner_tags'].apply(map_labels)

    # Drop rows with missing or unprocessable tags (if necessary)
    df = df.dropna(subset=['ner_tags']).reset_index(drop=True)

    return Dataset.from_pandas(df)

# Build `id_to_label_mapping` for Persian
id_to_label_mapping_persian = {}
for tagsnames, tagslabels in zip(persian_dataset['train']['ner_tags_names'], persian_dataset['train']['ner_tags']):
    if tagsnames is not None:
        for label, label_id in zip(tagsnames, tagslabels):
            id_to_label_mapping_persian[label_id] = label

# Preprocess Persian and English datasets
preprocessed_persian_train = preprocess_dataset(
    persian_dataset, language='Persian', split='train',
    id_to_label_mapping=id_to_label_mapping_persian, general_mapping=general_mapping
)
preprocessed_persian_test = preprocess_dataset(
    persian_dataset, language='Persian', split='test',
    id_to_label_mapping=id_to_label_mapping_persian, general_mapping=general_mapping
)
preprocessed_english_train = preprocess_dataset(
    english_dataset, language='English', split='train',
    id_to_label_mapping=id_to_label_mapping_english, general_mapping=general_mapping
)
preprocessed_english_test = preprocess_dataset(
    english_dataset, language='English', split='test',
    id_to_label_mapping=id_to_label_mapping_english, general_mapping=general_mapping
)

# Merge the two datasets
merged_train = Dataset.from_pandas(
    pd.concat(
        [preprocessed_persian_train.to_pandas(), preprocessed_english_train.to_pandas()],
        ignore_index=True
    )
)

merged_test = Dataset.from_pandas(
    pd.concat(
        [preprocessed_persian_test.to_pandas(), preprocessed_english_test.to_pandas()],
        ignore_index=True
    )
)

# Save the merged datasets
merged_dataset = DatasetDict({"train": merged_train.select_columns(["tokens", "ner_tags"]), "test": merged_test.select_columns(["tokens", "ner_tags"])})
merged_train.save_to_disk("./merged_train_dataset")
merged_test.save_to_disk("./merged_test_dataset")

print(f"Train dataset has {len(merged_train)} samples.")
print(f"Test dataset has {len(merged_test)} samples.")


Saving the dataset (0/1 shards):   0%|          | 0/40425 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/6749 [00:00<?, ? examples/s]

Train dataset has 40425 samples.
Test dataset has 6749 samples.


In [ ]:
merged_dataset['train'].features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=Value(dtype='int64', id=None), length=-1, id=None)}

<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px">

*بخش دوم:*
---
در مسائل Token Classification مانند NER، خروجی مدل‌ها (مانند RobertaForTokenClassification) باید با فرمت ورودی‌ها هماهنگ شود. این هماهنگی یا Alignment به دلایل زیر ضروری است:

* وجود Subword Tokenization: مدل‌هایی مثل xlm-Roberta از توکن‌سازی زیرواژه‌ای استفاده می‌کنند، که ممکن است یک کلمه را به چندین زیرواژه تقسیم کند. برای مثال، کلمه‌ای مانند unbelievable ممکن است به [un, ##believable] تبدیل شود. بنابراین برچسب‌دهی باید با این توکن‌های زیرواژه‌ای سازگار شود.

* حفظ سازگاری با برچسب‌های اصلی: از آنجا که داده‌ها برچسب‌دهی شده‌اند (مثلاً B_LOC, I_LOC)، باید مطمئن شویم که این برچسب‌ها به درستی به زیرواژه‌ها نگاشت شوند.

* افزایش دقت مدل: عدم هماهنگی بین ورودی‌ها و خروجی‌ها می‌تواند باعث خطا و کاهش دقت مدل شود

---

راه‌حل پیشنهادی:

استفاده از Align Labeling: این امکان می‌دهد تا برچسب‌های اصلی (label) را به توکن‌های زیرواژه‌ای مدل تطبیق دهیم.

* برای هر توکن اصلی، تطبیق دادن تمامی زیرواژه‌های آن توکن را با برچسب اصلی .

* علامت‌گذاری زیرواژه‌هایی که برچسبی ندارند (مانند padding یا special tokens) به عنوان O یا برچسب خنثی .

---

مراحل آموزش مدل:

1. تعریف مدل: استفاده از کلاس RobertaForTokenClassification و مشخص کردن تعداد برچسب‌ها (num_labels).

2. آموزش مدل:
    * استفاده از داده‌های پردازش‌شده (با Alignment).
    * استفاده از یک تابع خطا (مانند cross-entropy) برای محاسبه loss.

3. ارزیابی مدل

In [ ]:
from transformers import XLMRobertaTokenizer, XLMRobertaModel, TrainingArguments, Trainer, XLMRobertaTokenizerFast
from torch import nn
from datasets import load_from_disk

# Load the tokenizer and model
tokenizer = XLMRobertaTokenizerFast.from_pretrained("xlm-roberta-base")

def tokenize_and_align_labels(examples):
    print(f"Keys in examples: {list(examples.keys())}")

    # Tokenize the tokens
    tokenized_inputs = tokenizer(
        examples["tokens"],  # Use "tokens" as the input text
        truncation=True,
        padding="max_length",
        max_length=64,
        is_split_into_words=True  # Since tokens are provided as words
    )

    labels = []
    for i, label in enumerate(examples["ner_tags"]):  # Use "ner_tags" for labels
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to words
        label_ids = []
        previous_word_id = None
        for word_id in word_ids:
            if word_id is None:
                label_ids.append(-100)  # Ignored index
            elif word_id != previous_word_id:
                label_ids.append(label[word_id])  # Use the corresponding label
            else:
                label_ids.append(-100)  # Ignored for sub-tokens
            previous_word_id = word_id
        labels.append(label_ids)

    # Add the aligned labels to the tokenized inputs
    tokenized_inputs["labels"] = labels
    return tokenized_inputs


tokenized_datasets = merged_dataset.map(tokenize_and_align_labels, batched=True)

# Define the model
class XLMRobertaForTokenClassification(nn.Module):
    def __init__(self, num_labels):
        super(XLMRobertaForTokenClassification, self).__init__()
        self.backbone = XLMRobertaModel.from_pretrained("xlm-roberta-base")
        self.classifier = nn.Linear(self.backbone.config.hidden_size, num_labels)
        self.backbone.config.gradient_checkpointing = True

    def forward(self, input_ids, attention_mask, labels=None):
        outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        logits = self.classifier(outputs.last_hidden_state)
        loss = None
        if labels is not None:
            loss_fct = nn.CrossEntropyLoss(ignore_index=-100)
            loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return {"loss": loss, "logits": logits}

num_labels = len(set(label for sublist in merged_dataset["train"]["ner_tags"] for label in sublist))


Map:   0%|          | 0/40425 [00:00<?, ? examples/s]

Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples

Map:   0%|          | 0/6749 [00:00<?, ? examples/s]

Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']
Keys in examples: ['tokens', 'ner_tags']


In [ ]:
# Initialize the model
model = XLMRobertaForTokenClassification(num_labels=num_labels)

# Training arguments
training_args = TrainingArguments(
    output_dir="./Roberta-fa-en-ner",
    log_level="error",
    num_train_epochs=1,
    # gradient_checkpointing=True,
    eval_accumulation_steps=10,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    seed=42,
    logging_strategy="steps",
    eval_strategy="steps",
    save_steps=1e6,
    weight_decay=0.01,
    disable_tqdm=False,
    logging_steps=1e6,
    eval_steps=400,
    push_to_hub=False
)


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
import numpy as np

# Function to compute metrics
def compute_metrics(pred):
    predictions, labels = pred
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (-100) from predictions and labels
    true_labels = []
    true_predictions = []
    for label, prediction in zip(labels, predictions):
        filtered_labels = []
        filtered_predictions = []
        for l, p in zip(label, prediction):
            if l != -100:
                filtered_labels.append(l)
                filtered_predictions.append(p)
        true_labels.extend(filtered_labels)
        true_predictions.extend(filtered_predictions)

    # Compute metrics
    precision = precision_score(true_labels, true_predictions, average="weighted")
    recall = recall_score(true_labels, true_predictions, average="weighted")
    f1 = f1_score(true_labels, true_predictions, average="weighted")
    accuracy = accuracy_score(true_labels, true_predictions)

    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

# Add the compute_metrics function to the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

# Evaluate the model
metrics = trainer.evaluate()
print("Evaluation Metrics:")
print(metrics)


<ipython-input-10-9804ac940c57>:36: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: seyyed-msl82 (seyyed-msl82-university-of-tehran). Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Evaluation Metrics:
{'eval_loss': 0.0905294194817543, 'eval_accuracy': 0.9734490681644116, 'eval_precision': 0.9743064873971822, 'eval_recall': 0.9734490681644116, 'eval_f1': 0.9736294743400122, 'eval_runtime': 22.0638, 'eval_samples_per_second': 305.886, 'eval_steps_per_second': 2.402, 'epoch': 1.0}


<div dir="rtl" style="font-family: Vazir; width: 85%; font-size: 16px">

*بخش سوم:*
---
در مسائل مربوط به Token Classification مانند Named Entity Recognition (NER)، متریک Accuracy به تنهایی کافی نیست، زیرا:

* عدم توزیع یکنواخت برچسب‌ها: بسیاری از توکن‌ها در جملات معمولاً دارای برچسب خنثی (O) هستند. اگر مدل به طور مداوم فقط برچسب O را پیش‌بینی کند، Accuracy بالایی خواهد داشت، اما عملکرد مدل روی دسته‌های اصلی (مانند B_LOC یا I_PER) ارزیابی نمی‌شود.

* بی‌توجهی به تعادل بین Precision و Recall: Accuracy نمی‌تواند تناسب پیش‌بینی‌های صحیح مثبت (True Positives) را با پیش‌بینی‌های غلط (False Positives) یا پیش‌بینی‌های از دست رفته (False Negatives) ارزیابی کند.

برای مسائل متعادل‌تر و دقیق‌تر، معیارهای Precision, Recall و F1-Score ضروری هستند:

* Precision: دقت در پیش‌بینی برچسب‌های مثبت.

* Recall: توانایی مدل در پیدا کردن تمام نمونه‌های مثبت.

* F1-Score: میانگین هماهنگ Precision و Recall برای ایجاد توازن بین این د

In [ ]:
general_mapping_index_to_id = {
    0: 'O',
    1: 'B_PER',
    2: 'I_PER',
    3: 'B_LOC',
    4: 'I_LOC',
    5: 'B_ORG',
    6: 'I_ORG',
    7: 'B_MISC',
    8: 'I_MISC',
    9: 'B_EVE',
    10: 'I_EVE',
    11: 'B_PRO',
    12: 'I_PRO',
    13: 'B_FAC',
    14: 'I_FAC',
    15: 'B_MON',
    16: 'I_MON',
    17: 'B_DAT',
    18: 'I_DAT',
    19: 'B_TIM',
    20: 'I_TIM',
    21: 'B_PCT',
    22: 'I_PCT'
}

# Display output for a sample sentence
sample_text = "I am Seyyed Reza Moslemi and love Milan city."  # Example input
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, max_length=128).to(device)
outputs = model(**inputs)
predicted_logits = outputs["logits"]
predicted_labels = np.argmax(predicted_logits.cpu().detach().numpy(), axis=2)

print(len(predicted_labels[0]))

# Map predictions back to label names
predicted_entities = [general_mapping_index_to_id[label] for label in predicted_labels[0] if label != -100]

print(f"Input Sentence: {sample_text}")
print(f"Predicted Labels: {predicted_entities[1:-1]}")

17
Input Sentence: I am Seyyed Reza Moslemi and love Milan city.
Predicted Labels: ['O', 'O', 'B_PER', 'I_PER', 'I_PER', 'I_PER', 'I_PER', 'I_PER', 'I_PER', 'I_PER', 'O', 'O', 'B_LOC', 'O', 'O']
